In [ ]:
import io
import math
import random
import itertools
import warnings

import cypari2
from cysignals.alarm import alarm, AlarmInterrupt

from matplotlib import pyplot as plt
from collections import Counter
from tqdm.auto import tqdm, trange

from pyecsca.misc.cfg import TemporaryConfig
from pyecsca.misc.utils import TaskExecutor
from pyecsca.ec.mod import mod, RandomModAction
from pyecsca.ec.point import Point
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.params import load_params_ectester, get_params
from pyecsca.ec.mult import LTRMultiplier, RTLMultiplier, ScalarMultiplicationAction
from pyecsca.ec.context import local, DefaultContext
from countermeasures import GroupScalarRandomization, AdditiveSplitting, MultiplicativeSplitting, EuclideanSplitting, BrumleyTuveri
from pyecsca.ec.formula import AdditionFormula
from pyecsca.ec.mod import Mod, mod
from pyecsca.ec.mult import ScalarMultiplier, ScalarMultiplicationAction
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.point import Point
from countermeasures import *
from public import public
from abc import ABC, abstractmethod
from typing import Optional
# %matplotlib ipympl

In [ ]:
# copy from simulation notebook

def generate_scalars_mod3(rem, samples):
    scalars = []
    while True:
        scalar = random.randint(0, params3n.full_order)
        if scalar % 3 == rem:
            scalars.append(scalar)
        if len(scalars) == samples:
            break
    return scalars

def test_3n(countermeasure, scalars):
    ctr = Counter()
    for k in tqdm(scalars, leave=False):
        mult.init(params3n, point3n)
        kP = mult.multiply(k).to_affine()
        mult.init(params3n, point3n)
        knP = mult.multiply(k + params3n.full_order).to_affine()
        mult.init(params3n, point3n)
        k2nP = mult.multiply(k + 2 * params3n.full_order).to_affine()

        countermeasure.init(params3n, point3n)
        res = countermeasure.multiply(k)
        aff = res.to_affine()
        if aff.equals(kP):
            ctr["k"] += 1
        elif aff.equals(knP):
            ctr["k + 1n"] += 1
        elif aff.equals(k2nP):
            ctr["k + 2n"] += 1
        else:
            ctr[aff] += 1
            
    print(ctr)
    for name, count in sorted(ctr.items()):
        print(f"{name}:\t{count}")

def test_3n_fixed_scalar(countermeasure, samples):
    test_3n(countermeasure, [key3n for _ in range(samples)])

def test_3n_random_scalar(countermeasure, samples):
    test_3n(countermeasure, [random.randint(0, params3n.full_order) for _ in range(samples)])

def test_3n_random_scalar_projected(countermeasure, samples):
    print("k = 0 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(0, samples))
    print()
    print("k = 1 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(1, samples))
    print()
    print("k = 2 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(2, samples))

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
ltr = LTRMultiplier(add, dbl, complete=False)
rtl = RTLMultiplier(add, dbl, complete=False)
mult = ltr
gsr = GroupScalarRandomization(mult)
gsr160 = GroupScalarRandomization(mult, 160)
asplit = AdditiveSplitting(mult)
msplit = MultiplicativeSplitting(mult)
esplit = EuclideanSplitting(mult,add)
bt = BrumleyTuveri(mult)

In [ ]:
key3n = 0x20959f2b437de1e522baf6d814911938157390d3ea5118660b852ab0d5387006
params3n = load_params_ectester(io.BytesIO(b"0xc381bb0394f34b5ed061c9107b66974f4d0a8ec89b9fe73b98b6d1368c7d974d,0x5ca6c5ee0a10097af291a8f125303fb1a3e35e8100411902245d691e0e5cb497,0x385a5a8bb8af94721f6fd10b562606d9b9df931f7fd966e96859bb9bd7c05836,0x4616af1898b92cac0f902a9daee24bbae63571cead270467c6a7886ced421f5e,0x34e896bdb1337e0ae5960fa3389fb59c2c8d6c7dbfd9aac33a844f8f98e433ef,0x412b3e5686fbc3ca4575edb0292232702ae721a7d4a230cc170a5561aa70e00f,0x01"), "projective")
bits3n = params3n.full_order.bit_length()
point3n = Point(X=mod(0x4a48addb2e471767b7cd0f6f1d4c27fe46f4a828fc20f950bd1f72c939b36a84, params3n.curve.prime),
                Y=mod(0x13384d38c353f862832c0f067e46a3e510bb6803c20745dfb31929f4a18d890d, params3n.curve.prime),
                Z=mod(1, params3n.curve.prime), model=coords)

## GSR + Add

In [ ]:
@public
class GSR_Add_1(ScalarMultiplierCountermeasure):
    r"""
    GSR with Add, option 1.
    [k+rn-s]+[s]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits: int = 32, add = None):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits = rand_bits
        self.add = add
        self.r = None
        self.s = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = int(Mod.random(1 << self.rand_bits)) if self.r is None else self.r
            order = self.params.order
            s = Mod.random(order)  if self.s is None else self.s
            gsr_scalar = scalar + r * order
            m1 = self.mult.multiply(int(gsr_scalar-s))
            m2 = self.mult.multiply(int(s))
            if self.add is None:
                res = self.mult._add(m1, m2)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, m1, m2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
@public
class GSR_Add_2(ScalarMultiplierCountermeasure):
    r"""
    GSR with Add, option 2.
    [k-s+r_1*n]+[s+r_2*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits: int = 32, add = None):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits = rand_bits
        self.add = add
        self.r1 = None
        self.r2 = None
        self.s = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r1 = int(Mod.random(1 << self.rand_bits))  if self.r1 is None else self.r1
            r2 = int(Mod.random(1 << self.rand_bits))  if self.r2 is None else self.r2
            order = self.params.order
            s = Mod.random(order)  if self.s is None else self.s
            add_scalar = scalar - s
            
            m1 = self.mult.multiply(int(add_scalar+r1*order))
            m2 = self.mult.multiply(int(s+r2*order))
            if self.add is None:
                res = self.mult._add(m1, m2)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, m1, m2, **self.params.curve.parameters
                )[0]
            return action.exit(res)

In [ ]:
ga1 = GSR_Add_1(mult)
ga2 = GSR_Add_2(mult)

In [ ]:
test_3n_random_scalar(ga1, 10)

In [ ]:
test_3n_random_scalar(ga2, 10)

In [ ]:
test_3n_random_scalar_projected(ga1,10)

In [ ]:
test_3n_random_scalar_projected(ga2, 10)

In [ ]:
test_3n_fixed_scalar(ga1,10)

In [ ]:
test_3n_fixed_scalar(ga2, 10)

In [ ]:
gsrbits = 32
ga1 = GSR_Add_1(mult)
ga1.r1 = int(Mod.random(1 << gsrbits))
ga1.r2 = int(Mod.random(1 << gsrbits))
ga2 = GSR_Add_2(mult)
ga2.r1 = int(Mod.random(1 << gsrbits))
ga2.r2 = int(Mod.random(1 << gsrbits))

In [ ]:
test_3n_random_scalar_projected(ga1,10)

In [ ]:
test_3n_random_scalar_projected(ga2,300)

In [ ]:
test_3n_random_scalar_projected(ga2,300)

In [ ]:
order = params3n.order
ga1 = GSR_Add_1(mult)
ga1.s = Mod.random(order)
ga2 = GSR_Add_2(mult)
ga2.s = Mod.random(order)

In [ ]:
test_3n_random_scalar_projected(ga1,300)

In [ ]:
test_3n_random_scalar_projected(ga2,300)

## GSR + Mult

In [ ]:
@public
class GSR_Mult_1(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 1.
    [ks^(-1)+rn][s]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.r = None
        self.s = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = Mod.random(1 << self.rand_bits_gsr)  if self.r is None else self.r
            s = Mod.random(1 << self.rand_bits_mult)  if self.s is None else self.s
            
            S = self.mult.multiply(int(s))
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(int(s), order).inverse()
            ks_inv_gsr = int(ks_inv) + int(r)*order
            return action.exit(self.mult.multiply(int(ks_inv_gsr)))

@public
class GSR_Mult_2(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 2.
    [ks^(-1)+r_1*n][s+r_2*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.s = None
        self.r1 = None
        self.r2 = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r1 = Mod.random(1 << self.rand_bits_gsr)  if self.r1 is None else self.r1
            r2 = Mod.random(1 << self.rand_bits_gsr)  if self.r2 is None else self.r2
            s = Mod.random(1 << self.rand_bits_mult)  if self.s is None else self.s
            s_gsr = s+ r2*order 
            
            S = self.mult.multiply(int(s_gsr))
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(int(s_gsr), order).inverse()
            ks_inv_gsr = int(ks_inv) + int(r1)*order
            return action.exit(self.mult.multiply(int(ks_inv_gsr)))
        

@public
class GSR_Mult_3(ScalarMultiplierCountermeasure):
    r"""
     GSR with Mult, option 2.
    [ks^(-1)][s+r*n]
    """

    rand_bits: int

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, rand_bits_mult: int = 32):
        """
        :param mult: The multiplier to use.
        :param rand_bits: How many random bits to sample.
        """
        super().__init__(mult)
        self.rand_bits_gsr = rand_bits_gsr
        self.rand_bits_mult = rand_bits_mult
        self.r = None
        self.s = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = Mod.random(1 << self.rand_bits_gsr) if self.r is None else self.r
            s = Mod.random(1 << self.rand_bits_mult) if self.s is None else self.s
            s_gsr = s+ r*order 
            
            S = self.mult.multiply(int(s_gsr))
            self.mult.init(self.params, S)
            
            ks_inv = scalar * mod(int(s_gsr), order).inverse()
            return action.exit(self.mult.multiply(int(ks_inv)))

In [ ]:
gm1 = GSR_Mult_1(mult)
gm2 = GSR_Mult_2(mult)
gm3 = GSR_Mult_3(mult)

In [ ]:
test_3n_random_scalar(gm1, 10)

In [ ]:
test_3n_random_scalar(gm2, 10)

In [ ]:
test_3n_random_scalar(gm3, 10)

In [ ]:
test_3n_random_scalar_projected(gm1,10)

In [ ]:
test_3n_random_scalar_projected(gm2,10)

In [ ]:
test_3n_random_scalar_projected(gm3,10)

In [ ]:
test_3n_fixed_scalar(gm1,10)

In [ ]:
test_3n_fixed_scalar(gm2,10)

In [ ]:
test_3n_fixed_scalar(gm3,10)

In [ ]:
r = Mod.random(32)
gm1 = GSR_Mult_1(mult)
gm1.r = r
gm2 = GSR_Mult_2(mult)
gm2.r = r
gm3 = GSR_Mult_3(mult)
gm3.r = r

In [ ]:
test_3n_random_scalar_projected(gm1,300)

In [ ]:
test_3n_random_scalar_projected(gm2,300)

In [ ]:
test_3n_random_scalar_projected(gm3,300)

In [ ]:
s = Mod.random(32)
gm1 = GSR_Mult_1(mult)
gm1.s = s
gm2 = GSR_Mult_2(mult)
gm2.s = s
gm3 = GSR_Mult_3(mult)
gm3.s = s

In [ ]:
test_3n_random_scalar_projected(gm1,300)

In [ ]:
test_3n_random_scalar_projected(gm2,300)

In [ ]:
test_3n_random_scalar_projected(gm3,300)

### GSR + Euclid

In [ ]:
@public
class GSR_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 1.
     k+rn = k1+k2s
    [k1]+[k2][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r = None
        self.s = None

    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )
        
    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r = Mod.random(1 << self.rand_bits_gsr)  if self.r is None else self.r
            gsr = int(scalar)+int(r)*int(order)
            s = Mod.random(1 << half_bits)  if self.s is None else self.s
            S = self.mult.multiply(int(s))
            k1 = gsr % int(s)
            k2 = gsr // int(s)
            T = self.mult.multiply(k1)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
        
@public
class GSR_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 2.
     k = k1+k2s
    [k1]+[k2][s+r*n]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r = None
        self.s = None
        
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r = Mod.random(1 << self.rand_bits_gsr)  if self.r is None else self.r
            s = Mod.random(1 << half_bits)  if self.s is None else self.s
            S = self.mult.multiply(int(s)+int(r)*order)
            k1 = int(scalar) % int(s)
            k2 = int(scalar) // int(s)
            T = self.mult.multiply(k1)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
@public
class GSR_Eucl_3(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 3.
     k = k1+k2s
    [k1+r1*n]+[k2+r2*n][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r1 = None
        self.r2 = None
        self.s = None
    
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = Mod.random(1 << self.rand_bits_gsr)  if self.r1 is None else self.r1
            r2 = Mod.random(1 << self.rand_bits_gsr)  if self.r2 is None else self.r2
            s = Mod.random(1 << half_bits)  if self.s is None else self.s
            S = self.mult.multiply(int(s))
            k1 = int(scalar) % int(s)
            k2 = int(scalar) // int(s)
            T = self.mult.multiply(k1+int(r1)*order)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2+int(r2)*order)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        
@public
class GSR_Eucl_4(ScalarMultiplierCountermeasure):
    r"""
     GSR with Eucl, option 4.
     k = k1+k2s
    [k1+r1*n]+[k2+r2*n][s+r3*n]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, rand_bits_gsr: int = 32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_gsr = rand_bits_gsr
        self.r1 = None
        self.r2 = None
        self.r3 = None
        self.s = None
        
    def init(self, params: DomainParameters, point: Point):
        self.params = params
        self.point = point
        self.mult.init(
            self.params,
            self.point,
            bits=params.full_order.bit_length() + self.rand_bits_gsr,
        )

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = Mod.random(1 << self.rand_bits_gsr)  if self.r1 is None else self.r1
            r2 = Mod.random(1 << self.rand_bits_gsr)  if self.r2 is None else self.r2
            r3 = Mod.random(1 << self.rand_bits_gsr)  if self.r3 is None else self.r3
            s = Mod.random(1 << half_bits) if self.s is None else self.s
            S = self.mult.multiply(int(s)+int(r3)*order)
            k1 = int(scalar) % int(s)
            k2 = int(scalar) // int(s)
            T = self.mult.multiply(k1+int(r1)*order)
            self.mult.init(self.params, S)
            R = self.mult.multiply(k2+int(r2)*order)
            if self.add is None:
                res = self.mult._add(R, T)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, T, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        


In [ ]:
ge1 = GSR_Eucl_1(mult)
ge2 = GSR_Eucl_2(mult)
ge3 = GSR_Eucl_3(mult)
ge4 = GSR_Eucl_4(mult)

In [ ]:
test_3n_random_scalar(ge1, 10)

In [ ]:
test_3n_random_scalar(ge2, 10)

In [ ]:
test_3n_random_scalar(ge3, 10)

In [ ]:
test_3n_random_scalar(ge4, 10)

In [ ]:
test_3n_random_scalar_projected(ge1,10)

In [ ]:
test_3n_random_scalar_projected(ge2,10)

In [ ]:
test_3n_random_scalar_projected(ge3,1000)

In [ ]:
test_3n_random_scalar_projected(ge4,1000)

In [ ]:
test_3n_fixed_scalar(ge1,1000)

In [ ]:
test_3n_fixed_scalar(ge2,1000)

In [ ]:
test_3n_fixed_scalar(ge3,1000)

In [ ]:
test_3n_fixed_scalar(ge4,1000)

In [ ]:
half_bits = params3n.order.bit_length() // 2
s = Mod.random(1 << half_bits)
ge1 = GSR_Eucl_1(mult)
ge1.s = s
ge2 = GSR_Eucl_2(mult)
ge2.s = s
ge3 = GSR_Eucl_3(mult)
ge3.s = s
ge4 = GSR_Eucl_4(mult)
ge4.s = s

In [ ]:
test_3n_random_scalar_projected(ge1,1000)

In [ ]:
test_3n_random_scalar_projected(ge2,1000)

In [ ]:
test_3n_random_scalar_projected(ge3,1000)

In [ ]:
test_3n_random_scalar_projected(ge4,1000)

In [ ]:
bits = 32
r1 = Mod.random(1 << bits)
r2 = Mod.random(1 << bits)
r3 = Mod.random(1 << bits)

ge1 = GSR_Eucl_1(mult)
ge1.r1 = r1
ge1.r2 = r2
ge1.r3 = r3
ge2 = GSR_Eucl_2(mult)
ge2.r1 = r1
ge2.r2 = r2
ge2.r3 = r3
ge3 = GSR_Eucl_3(mult)
ge3.r1 = r1
ge3.r2 = r2
ge3.r3 = r3
ge4 = GSR_Eucl_4(mult)
ge4.r1 = r1
ge4.r2 = r2
ge4.r3 = r3

In [ ]:
test_3n_random_scalar_projected(ge1,1000)

In [ ]:
test_3n_random_scalar_projected(ge2,1000)

In [ ]:
test_3n_random_scalar_projected(ge3,1000)

In [ ]:
test_3n_random_scalar_projected(ge4,1000)

### Add+Mul

In [ ]:
@public
class Add_Mul_1(ScalarMultiplierCountermeasure):
    r"""
     Add with Mul, option 1.
    [(k-r)s1^(-1)][s1]+[rs2^(-1)][s2]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r = None
        self.s1 = None
        self.s2 = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = Mod.random(order) if self.r is None else self.r
            s1 = Mod.random(1 << self.rand_bits_mul) if self.s1 is None else self.s1
            s2 = Mod.random(1 << self.rand_bits_mul) if self.s2 is None else self.s2
            kr = scalar - r
            
            kr_inv = kr * mod(int(s1), order).inverse()
            r_inv = r * mod(int(s2), order).inverse()
            
            R = self.mult.multiply(int(s1))
            self.mult.init(self.params, R)
            R = self.mult.multiply(int(kr_inv))
            
            self.mult.init(self.params, self.point)
            S = self.mult.multiply(int(s2))
            self.mult.init(self.params, S)
            S = self.mult.multiply(int(r_inv))
            if self.add is None:
                res = self.mult._add(R, S)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, S, **self.params.curve.parameters
                )[0]
            return action.exit(res)

@public
class Add_Mul_2(ScalarMultiplierCountermeasure):
    r"""
     Add with Mul, option 2.
    ([ks^(-1)-r]+[r])[s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r = None
        self.s = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            order = self.params.order
            r = Mod.random(order) if self.r is None else self.r
            s = Mod.random(1 << self.rand_bits_mul) if self.s is None else self.s
            
            ks_inv = scalar * mod(int(s), order).inverse()
            
            S = self.mult.multiply(int(s))
            self.mult.init(self.params, S)
            
            R = self.mult.multiply(int(ks_inv-r))
            S = self.mult.multiply(int(r))

            if self.add is None:
                res = self.mult._add(R, S)  # noqa: This is OK.
            else:
                res = self.add(
                    self.params.curve.prime, R, S, **self.params.curve.parameters
                )[0]
            return action.exit(res)

In [ ]:
am1 = Add_Mul_1(mult)
am2 = Add_Mul_2(mult)

In [ ]:
test_3n_random_scalar(am1, 10)

In [ ]:
test_3n_random_scalar(am2, 1000)

In [ ]:
test_3n_random_scalar_projected(am1, 1000)

In [ ]:
test_3n_random_scalar_projected(am2, 1000)

In [ ]:
test_3n_fixed_scalar(am1,1000)

In [ ]:
test_3n_fixed_scalar(am2,1000)

### Add + Eucl

In [ ]:
      
@public
class Add_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     Add with Eucl, option 1.
     k-r = k1+k2s1
     r = l1+l2s2
    [k1]+[k2][s1]+[l1]+[l2][s2]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.r = None
        self.s1 = None
        self.s2 = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r = Mod.random(order) if self.r is None else self.r
            s1 = Mod.random(1 << half_bits) if self.s1 is None else self.s1
            s2 = Mod.random(1 << half_bits) if self.s2 is None else self.s2
            k_r = int(scalar-r)
            k1 = k_r % int(s1)
            k2 = k_r // int(s1)
            
            l1 = int(r) % int(s2)
            l2 = int(r) // int(s2)
            
            
            S1 = self.mult.multiply(int(s1))
            S2 = self.mult.multiply(int(s2))
            K1 = self.mult.multiply(int(k1))
            L1 = self.mult.multiply(int(l1))
            
            self.mult.init(self.params, S1)
            K2 = self.mult.multiply(int(k2))
            self.mult.init(self.params, S2)
            L2 = self.mult.multiply(int(l2))
            
            if self.add is None:
                res = self.mult._add(K1, K2)
                res = self.mult._add(res, L1)
                res = self.mult._add(res, L2)
            else:
                res = self.add(
                    self.params.curve.prime, K1, K2, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, L1, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, L2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        

          
@public
class Add_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     Add with Eucl, option 2.
     k = k1+k2s
    [k1-r1]+[r1]+([k2-r2]+[r2])[s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.r1 = None
        self.r2 = None
        self.s = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            r1 = Mod.random(order) if self.r1 is None else self.r1
            r2 = Mod.random(order) if self.r2 is None else self.r2
            s = Mod.random(1 << half_bits) if self.s is None else self.s
            k1 = int(scalar) % int(s)
            k2 = int(scalar) // int(s)
            k1r1 = k1-r1
            k2r2 = k2-r2
            
            
            S = self.mult.multiply(int(s))
            KR1 = self.mult.multiply(int(k1r1))
            R1 = self.mult.multiply(int(r1))
            
            self.mult.init(self.params, S)
            KR2 = self.mult.multiply(int(k2r2))
            R2 = self.mult.multiply(int(r2))
            
            if self.add is None:
                res = self.mult._add(KR1, R1)
                res = self.mult._add(res, KR2)
                res = self.mult._add(res, R2)
            else:
                res = self.add(
                    self.params.curve.prime, KR1, R1, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, KR2, **self.params.curve.parameters
                )[0]
                res = self.add(
                    self.params.curve.prime, res, R2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        


In [ ]:
ae1 = Add_Eucl_1(mult)
ae2 = Add_Eucl_2(mult)


In [ ]:
test_3n_random_scalar(ae1, 10)

In [ ]:
test_3n_random_scalar(ae2, 1000)

In [ ]:
test_3n_random_scalar_projected(ae1, 1000)

In [ ]:
test_3n_random_scalar_projected(ae2, 1000)

In [ ]:
test_3n_fixed_scalar(ae1,1000)

In [ ]:
test_3n_fixed_scalar(ae2,1000)

In [ ]:
          
@public
class Mul_Eucl_1(ScalarMultiplierCountermeasure):
    r"""
     Mul with Eucl, option 1.
     kr^(-1) = k1+k2s
    ([k1]+[k2][s])[r]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.r = None
        self.s = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            s = Mod.random(order) if self.s is None else self.s
            r = Mod.random(1 << self.rand_bits_mul) if self.r is None else self.r
            
            kr_inv = scalar * mod(int(r), order).inverse()
            
            k1 = int(kr_inv) % int(s)
            k2 = int(kr_inv) // int(s)
            
            R = self.mult.multiply(int(r))
            self.mult.init(self.params, R)
            
            K1 = self.mult.multiply(int(k1))
            
            S = self.mult.multiply(int(s))
            self.mult.init(self.params, S)
            K2 = self.mult.multiply(int(k2))
            
            if self.add is None:
                res = self.mult._add(K1, K2)
            else:
                res = self.add(
                    self.params.curve.prime, K1, K2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        

                 
@public
class Mul_Eucl_2(ScalarMultiplierCountermeasure):
    r"""
     Mul with Eucl, option 2.
    [k1r1^(-1)][r1]+[k2r2^(-1)][r2][s]
    """

    add: Optional[AdditionFormula]

    def __init__(self, mult: ScalarMultiplier,rand_bits_mul=32, add: Optional[AdditionFormula] = None):
        """
        :param mult: The multiplier to use.
        :param add: Addition formula to use, if None, the formula from the multiplier is used.
        """
        super().__init__(mult)
        self.add = add
        self.rand_bits_mul = rand_bits_mul
        self.s = None
        self.r1 = None
        self.r2 = None

    def multiply(self, scalar: int) -> Point:
        if self.params is None or self.point is None:
            raise ValueError("Not initialized.")
        with ScalarMultiplicationAction(self.point, self.params, scalar) as action:
            half_bits = self.params.order.bit_length() // 2
            order = self.params.order
            s = Mod.random(order) if self.s is None else self.s
            
            r1 = Mod.random(1 << self.rand_bits_mul) if self.r1 is None else self.r1
            r2 = Mod.random(1 << self.rand_bits_mul) if self.r2 is None else self.r2
            
            k1 = int(scalar) % int(s)
            k2 = int(scalar) // int(s)
            
            kr1_inv = k1 * mod(int(r1), order).inverse()
            kr2_inv = k2 * mod(int(r2), order).inverse()
            
            R1 = self.mult.multiply(int(r1))
            S = self.mult.multiply(int(s))
            
            self.mult.init(self.params, R1)
            R1 = self.mult.multiply(int(kr1_inv))
            
            self.mult.init(self.params, S)
            R2 = self.mult.multiply(int(r2))
            
            self.mult.init(self.params, R2)
            R2 = self.mult.multiply(int(kr2_inv))
            
            
            if self.add is None:
                res = self.mult._add(R1, R2)
            else:
                res = self.add(
                    self.params.curve.prime, R1, R2, **self.params.curve.parameters
                )[0]
            return action.exit(res)
        


In [ ]:
me1 = Mul_Eucl_1(mult)
me2 = Mul_Eucl_2(mult)

In [ ]:
test_3n_random_scalar(me1, 10)

In [ ]:
test_3n_random_scalar(me2, 1000)

In [ ]:
test_3n_random_scalar_projected(me1, 1000)

In [ ]:
test_3n_random_scalar_projected(me2, 1000)

In [ ]:
test_3n_fixed_scalar(me1,1000)

In [ ]:
test_3n_fixed_scalar(me2,1000)